## Сложные типы данных

### JSON

JSON - JavaScript Object Notation. JSON - де-факто стандарт для хранения данных в виде key-value пар. 

Рассмотрим простой пример:

```sql
CREATE TABLE orders (
 ID serial NOT NULL PRIMARY KEY,
 info json NOT NULL
);

INSERT INTO orders (info)
VALUES
 (
 '{ "customer": "John Doe", "items": {"product": "Beer","qty": 6}}'
 ),
 (
 '{ "customer": "Lily Bush", "items": {"product": "Diaper","qty": 24}}'
 ),
 (
 '{ "customer": "Josh William", "items": {"product": "Toy Car","qty": 1}}'
 ),
 (
 '{ "customer": "Mary Clark", "items": {"product": "Toy Train","qty": 2}}'
 );
```

Запрос данных:

```sql
SELECT
 info
FROM
 orders;
```

Postgres вовзращает ответ в виде типа JSON. Для работы с ним есть 2 специальных оператора -> и ->>

-> возвращает результат в виде JSON-объекта
->> вовзращает результат в виде текста

Получить имена всех покупателей:
```sql
SELECT
 info ->> 'customer' AS customer
FROM
 orders;
```

т.к. оператор -> возвращает JSON-объект, то к его результату можно снова применять оператор -> и ->>. Пример:

``` sql
SELECT
 info -> 'items' ->> 'product' as product
FROM
 orders
ORDER BY
 product;
```

Еще пример. Поиск людей, которые купили 2 продукта:

```sql
SELECT
 info ->> 'customer' AS customer,
 info -> 'items' ->> 'product' AS product
FROM
 orders
WHERE
 CAST (
 info -> 'items' ->> 'qty' AS INTEGER
 ) = 2
```

## Практика 3

Создайте таблицу orders (скрипт выше в лекции). Выведите общее количество  item'ов во всех заказах

## Массивы

Массив - это коллекция элементов. В одной колонке Вы можете хранить несколько аттрибутов одного типа. Пример:

```sql
CREATE TABLE contacts (
 id serial PRIMARY KEY,
 name VARCHAR (100),
 phones TEXT []
);
```

### Вставка данных:
```sql
INSERT INTO contacts (name, phones)
VALUES
 (
 'John Doe',
 ARRAY [ '(408)-589-5846',
 '(408)-589-5555' ]
 );
```

или


```sql
INSERT INTO contacts (name, phones)
VALUES
 (
 'John Doe',
 '{ "(408)-589-5846", "(408)-589-5555" }'
 );
```

### Выборка данных:

```sql
SELECT
 name,
 phones
FROM
 contacts;
```

### Запрос конкретного элемента массива:

```sql
SELECT
 name,
 phones [ 1 ]
FROM
 contacts;
```

Индексы начинаются с 1.

###  Модификация данных

Обновление элемента
```sql
UPDATE contacts
SET phones [ 2 ] = '(408)-589-5843'
WHERE
 ID = 3;
```

Обновдение всего массива
```sql
UPDATE contacts
SET phones = '{"(408)-589-5843"}'
WHERE
 ID = 3;
```

### Поиск элемента

```sql 
SELECT
 name,
 phones
FROM
 contacts
WHERE
 '(408)-589-5555' = ANY (phones);
```

### Expand

unnest - превращает массив в набор строк

```sql
SELECT
 name,
 unnest(phones)
FROM
 contacts;
```

## Практика 4

Выведите сколько раз встречается какой специлаьный аттрибут (special_feature) у фильма 

## Представления

View - это именнованные запросы, которые помогают сделать представление(именно вид) данных, лежащий в таблицах PostgreSQL. View основывается на одной или нескольких базовых таблицах. Удобны для часто используемых запросов.

View (кроме materialized view) не хранят данные.

*Создание*

```sql
CREATE VIEW view_name AS query;
```

Пример с информацией о покупателях:

```sql
 SELECT cu.customer_id AS id,
    cu.first_name || ' ' || cu.last_name AS name,
    a.address,
    a.postal_code AS "zip code",
    a.phone,
    city.city,
    country.country,
        CASE
            WHEN cu.activebool THEN 'active'
            ELSE ''
        END AS notes,
    cu.store_id AS sid
   FROM customer cu
     INNER JOIN address a USING (address_id)
     INNER JOIN city USING (city_id)
     INNER JOIN country USING (country_id);
```

и теперь для получения данных о покупателях можно использовать простой Select:

```sql
SELECT
 *
FROM
 customer_master;
```

*Изменение*

```sql
CREATE OR REPLACE view_name 
AS 
query
```

*Удаление*

```sql
DROP VIEW [ IF EXISTS ] view_name;
```

## Практика 5

Создайте view с колонками клиент (ФИО; email) и title фильма, который он брал в прокат последним

## Материализованное представление
Хранит результат запроса. Засчет этого доступ к информации происходит быстрее, но материализованное представление надо переодически обновлять

```sql
CREATE MATERIALIZED VIEW view_name
AS
query
WITH [NO] DATA;
```

WITH DATA - загрузить данные сразу, WITH NO DATA - позже

*Обновление*

```sql
REFRESH MATERIALIZED VIEW view_name;
```

*Удаление*

```sql
DROP MATERIALIZED VIEW view_name;
```

## Ускорение запросов: индексы

Ускорить запрос можно с помощью создания индексов. Индексы можно создавать на лету

<pre>
CREATE INDEX ON ratings(movieId);
</pre>

Результат:

<pre>
Time: 37427.672 ms (00:37.428)
</pre>

После того, как индекс создан - запросы начинают выполнятся бодрее, время сокращается в сотни раз
<pre>
CREATE INDEX ON ratings(movieId);
</pre>

Результат:

<pre>
CREATE INDEX
Time: 38493.878 ms (00:38.494)
</pre>

Выполним запрос ещё раз:

<pre>
SELECT
    movieId,
    COUNT(*) num_rating
FROM public.ratings
WHERE
    ratings.movieID > 100000
GROUP BY 1
LIMIT 10;
</pre>

Результат:

<pre>
 movieid | num_rating
---------+------------
  100001 |          2
  100003 |          6
  100006 |          6
  100008 |         28
  100010 |         88
  100013 |         18
  100015 |          4
  100017 |         50
  100032 |         30
  100034 |         64
(10 rows)

Time: 5.289 ms
</pre>


## Схема запроса

Оператор EXPLAIN демострирует этапы выполнения запроса и может быть использован для оптимизации.

<pre>
EXPLAIN
SELECT
    userId, COUNT(*) num_rating
FROM public.links
LEFT JOIN public.ratings
    ON links.movieid=ratings.movieid
GROUP BY 1
LIMIT 10;
</pre>

Результат:

<pre>
                                      QUERY PLAN
--------------------------------------------------------------------------------------
 Limit  (cost=1880431.03..1880431.13 rows=10 width=16)
   ->  HashAggregate  (cost=1880431.03..1880749.83 rows=31880 width=16)
         Group Key: ratings.userid
         ->  Hash Right Join  (cost=1323.47..1620188.15 rows=52048576 width=8)
               Hash Cond: (ratings.movieid = links.movieid)
               ->  Seq Scan on ratings  (cost=0.00..903196.76 rows=52048576 width=16)
               ->  Hash  (cost=750.43..750.43 rows=45843 width=8)

</pre>

## Домашнее задание

Работаем с запросом:

```sql
Для каждого пользователя подсчитайте сколько он брал в аренду фильмов со специальным аттрибутом Behind the Scenes
```

1. Написать этот запрос
2. Создать материализованное представление с этим запросом и обновить его
3. Сделать explain этому запросу. Опираясь на вывод explain описать по-русски что делает база данных для получения резульатов. Описание строк в explain смотри тут - https://use-the-index-luke.com/sql/explain-plan/postgresql/operations

In [ ]:
## Дополнительные материалы:

1. https://postgrespro.ru/docs/postgrespro/9.5/using-explain
2. https://habr.com/ru/post/203320/
3. 